In [ ]:
# Standard library
from pathlib import Path
from typing import TYPE_CHECKING, Any

# Third-party libraries
import mujoco
import numpy as np
from mujoco import viewer
from rich.console import Console

# Local libraries
from ariel.body_phenotypes.robogen_lite.config import (NUM_OF_FACES,
                                                       NUM_OF_ROTATIONS,
                                                       NUM_OF_TYPES_OF_MODULES)

from ariel.body_phenotypes.robogen_lite.constructor import \
    construct_mjspec_from_graph
from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import (
    HighProbabilityDecoder, save_graph_as_json)
from ariel.body_phenotypes.robogen_lite.modules.core import CoreModule

from ariel.simulation.environments.simple_tilted_world import TiltedFlatWorld
from ariel.utils.renderers import single_frame_renderer

from ariel.body_phenotypes.robogen_lite.decoders.visualize_tree import visualize_tree_from_graph

from networkx import DiGraph

In [ ]:
# Global constants
SCRIPT_NAME = "Initial"
CWD = Path.cwd()
DATA = Path(CWD / "__data__" / SCRIPT_NAME)
DATA.mkdir(exist_ok=True)
SEED = 40

# Global functions
console = Console()
RNG = np.random.default_rng(SEED)


In [ ]:
def generate_body() -> DiGraph:
    """generate random robot with HighProbabilityDecoder"""
    # System parameters
    num_modules = 20

    # "Type" probability space
    type_probability_space = RNG.random(
        size=(num_modules, NUM_OF_TYPES_OF_MODULES),
        dtype=np.float32,
    )

    # "Connection" probability space
    conn_probability_space = RNG.random(
        size=(num_modules, num_modules, NUM_OF_FACES),
        dtype=np.float32,
    )

    # "Rotation" probability space
    rotation_probability_space = RNG.random(
        size=(num_modules, NUM_OF_ROTATIONS),
        dtype=np.float32,
    )

    # Decode the high-probability graph
    hpd = HighProbabilityDecoder(num_modules)
    graph = hpd.probability_matrices_to_graph(
        type_probability_space,
        conn_probability_space,
        rotation_probability_space,
    )

    # Decode the high-probability graph
    hpd = HighProbabilityDecoder(num_modules)
    graph = hpd.probability_matrices_to_graph(
            type_probability_space,
            conn_probability_space,
            rotation_probability_space,
        )
    return graph

In [ ]:
def run(
    robot: CoreModule,
    *,
    with_viewer: bool = False,
) -> None:
    """Entry point."""
    # MuJoCo configuration
    viz_options = mujoco.MjvOption()  # visualization of various elements

    # Visualization of the corresponding model or decoration element
    viz_options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
    viz_options.flags[mujoco.mjtVisFlag.mjVIS_ACTUATOR] = True
    viz_options.flags[mujoco.mjtVisFlag.mjVIS_BODYBVH] = True

    # MuJoCo basics
    world = TiltedFlatWorld()

    # Set random colors for geoms
    for i in range(len(robot.spec.geoms)):
        robot.spec.geoms[i].rgba[-1] = 0.5

    # Spawn the robot at the world
    world.spawn(robot.spec)

    # Compile the model
    model = world.spec.compile()
    data = mujoco.MjData(model)

    # Save the model to XML
    xml = world.spec.to_xml()
    with (DATA / f"{SCRIPT_NAME}.xml").open("w", encoding="utf-8") as f:
        f.write(xml)

    # Number of actuators and DoFs
    console.log(f"DoF (model.nv): {model.nv}, Actuators (model.nu): {model.nu}")

    # Reset state and time of simulation
    mujoco.mj_resetData(model, data)

    # Render
    # single_frame_renderer(model, data, steps=10)

    # View
    if with_viewer:
        viewer.launch(model=model, data=data)

In [ ]:
delta = 10000 # just a random nr that bigger than 0.1
diversity = 0
nr_of_blocks = []
old_avg_b = 0
old_avg_h = 0
nr_of_hinges = []
learnability = 0 # maybe later
graph_list = []
while delta > 0.00001:
    graph = generate_body()
    graph_list.append(graph)
    graph = dict(graph.nodes)
    hinges = sum(graph[node]["type"] == "HINGE" for node in graph)
    blocks = sum(graph[node]["type"] == "BRICK" for node in graph)
    nr_of_blocks.append(blocks)
    nr_of_hinges.append(hinges)

    # skip the first 100
    if len(nr_of_blocks)<100:
        continue

    # go until we broke
    new_avg_b = np.average(nr_of_blocks)
    new_avg_h = np.average(nr_of_hinges)
    delta = max(abs(old_avg_b-new_avg_b),abs(old_avg_h-new_avg_h))

    if len(graph_list) >= 100000:
        break
    
    
    old_avg_b = new_avg_b
    old_avg_h = new_avg_h

print(f"Done!\nEnded with a delta of {max(abs(old_avg_b-new_avg_b),abs(old_avg_h-new_avg_h))}\nThe avg nr of blocks are: {new_avg_b}\nThe avg nr of hinges is: {new_avg_h}\nConcluded after {len(graph_list)} generated robots!")


In [ ]:
body = generate_body()

visualize_tree_from_graph(body)